In [1]:
#環境設置
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import time
driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver92") 

def organize_columns(df1):
    # 合併全部的 DataFrame
    try:
        df1 = pd.concat(df1, axis=0, ignore_index=True)
    except:
        df1.reset_index(drop=True, inplace=True)

    # 處理 column 2：館藏地
    c2 = [
        '分館/專室', '館藏地/室', '館藏室', '館藏地/館藏室', '館藏地', '典藏館', '館藏位置', '館藏地/區域',
        '典藏地名稱', '館藏地/館別', '館藏地(已外借/總數)'
    ]
    df1['c2'] = ''
    for c in c2:
        try:
            df1['c2'] += df1[c]
        except:
            pass

    # 處理 column 3：索書號
    c3 = ['索書號', '索書號/期刊合訂本卷期', '索書號 / 部冊號']
    df1['c3'] = ''
    for c in c3:
        try:
            df1['c3'] += df1[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = [
        '館藏位置(到期日期僅為期限，不代表上架日期)', '狀態/到期日', '目前狀態 / 到期日', '館藏狀態', '處理狀態',
        '狀態 (說明)', '館藏現況 說明', '目前狀態/預計歸還日期', '圖書狀況 / 到期日', '調閱說明', '借閱狀態',
        '狀態', '館藏狀態(月-日-西元年)', '圖書狀況', '現況/異動日', 'Unnamed: 24'
    ]
    df1['c4'] = ''
    for c in c4:
        try:
            df1['c4'] += df1[c]
        except:
            pass

    # 直接生成另一個 DataFrame
    df2 = pd.DataFrame()
    df2['圖書館'] = df1['圖書館']
    df2['館藏地'] = df1['c2']
    df2['索書號'] = df1['c3']
    df2['館藏狀態'] = df1['c4']
    df2['連結'] = df1['連結']

    # 遇到值為 NaN時，將前一列的值填補進來
    df2.fillna(method="ffill", axis=0, inplace=True)

    return df2


def accurately_find_table_and_read_it(table_position, table_position2=0):
    try:
        if not wait_for_element_present(table_position):
            print(f'找不到 {table_position}！')
            return
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table_innerHTML = soup.select(table_position)[table_position2]
        tgt = pd.read_html(str(table_innerHTML), encoding='utf-8')[0]
        # tgt['圖書館'], tgt['連結'] = org, driver.current_url
    except:
        return
    else:
        return tgt

def wait_for_element_present(driver, element_position, waiting_time=5, by=By.CSS_SELECTOR):
    try:
        element = WebDriverWait(driver, waiting_time).until(
            EC.presence_of_element_located((by, element_position)))
    except:
        return
    else:
        return element

def wait_for_element_clickable(driver, element_position, waiting_time=5, by=By.LINK_TEXT):
    try:
        time.sleep(0.3)
        element = WebDriverWait(driver, waiting_time).until(
            EC.element_to_be_clickable((by, element_position)))
    except:
        return
    else:
        return element

def wait_for_elements_present(driver, elements_position, waiting_time=5, by=By.CSS_SELECTOR):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    try:
        node_off()
        elements = WebDriverWait(driver, waiting_time).until(EC.presence_of_all_elements_located((by, elements_position)))
    except Exception as e:
        alert_exception_report(function, e)
        return False
    else:
        return elements

In [49]:
# primo_crawler()
# 臺大|政大|淡大|東吳

def primo_finding(driver, org, tcn): 
    sub_df_lst = []
    try:
        back = wait_for_element_clickable(
            driver, ".tab-header > prm-opac-back-button > button", 5, By.CSS_SELECTOR).click()
    except:
        back = None

    time.sleep(3)
    try:
        thelist = driver.find_elements_by_class_name(tcn) 
        if tcn == 'md-2-line.md-no-proxy._md':  # 如果是東吳或銘傳
            thelist = thelist[0:-2]

        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    now_url = driver.current_url
                    if org == "銘傳大學":
                        new_row = [org, where[_].text,
                                "沒有索書號喔QQ", a[0].text, now_url]
                    else:
                        new_row = [org, where[_].text,
                                a[4].text, a[0].text, now_url]
                    sub_df_lst.append(new_row)
                    break
                break
    except:
        pass
    return sub_df_lst

def primo_crawler(driver, org, url_front, ISBN ,url_behind, tcn):
    url = url_front + ISBN + url_behind
    primo_lst = []

    try:
        # 進入《館藏系統》頁面
        driver.get(url)

        try:  # 開始爬蟲
            time.sleep(3)
            editions = driver.find_elements_by_class_name("item-title")
            #editions = wait_for_elements_present(driver, 'md-primoExplore-theme', 20, By.CLASS_NAME)
            print("進入搜尋")
        except:
            print("nono")
        if org == "國立政治大學" or org == "國立清華大學": #不要讓台大找箭頭，太太太混亂了
            try: #找有沒有多個版本的箭頭
                edition_check = wait_for_element_clickable(driver, "neutralized-button.arrow-link-button.md-button.md-primoExplore-theme.md-ink-ripple", 20, By.CLASS_NAME)
                edition_check.click()
            except:
                edition_check = None
        else:
            edition_check = None
            
        if edition_check != None: #如果有箭頭，代表有多個版本，再爬一次版本！
            print("有多個版本QQ")
            time.sleep(3)
            editions = driver.find_elements_by_class_name("item-title")
        else:
            editions[0].click()
            print(edition_check)


        print(len(editions))
        if len(editions) > 1:  # 多個版本才要再跑迴圈(找不到叉叉代表不在最裡面，可知不是一個版本)
            print("進多個版本")
            
            for i in range(0, len(editions)):  # 有幾個版本就跑幾次，不管哪一層版本都適用
                print("版本" + str(i + 1))
                time.sleep(2)
                into = editions[i].click()
                print(editions[i].text)
                time.sleep(3)
                if org == "國立屏東科技大學" or org == "國立高雄餐旅大學":
                    primo_lst += primo_two_finding(driver, org)
                else:
                    primo_lst += primo_finding(driver, org, tcn)
                    print(primo_lst)
                try:
                    back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple")
                    back2.click()
                except:
                    print("下一個失敗")

        else:  # 如果只有一個版本(有叉叉的意思)，那前面已經click過了不能再做
            if org == "國立屏東科技大學" or org == "國立高雄餐旅大學":
                primo_lst += primo_two_finding(driver, org)
                print(primo_lst)
            else:
                print("進一個版本的primo_finding")
                primo_lst += primo_finding(driver, org, tcn)
                print(primo_lst)
                
    except:
        pass
    table = pd.DataFrame(primo_lst)
    table.rename(columns={0: '圖書館', 1: '館藏地', 2: '索書號', 3: '館藏狀態', 4: '連結'}, inplace = True)
    return table

print(primo_crawler(driver,
        '國立陽明交通大學',
        "https://nycu.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
        "9789573229131",
        "&tab=Everything&search_scope=MyInst_and_CI&vid=886UST_NYCU:886UST_NYCU&lang=zh-tw&offset=0",
        "layout-align-space-between-center.layout-row.flex-100"
    ))

進入搜尋
None
5
進多個版本
版本1
Empty DataFrame
Columns: []
Index: []


In [37]:
#元智大學
url = "http://lib.yzu.edu.tw/ajaxYZlib/Search/SearchNew.aspx"
ISBN = 9789861371955

table_lst = []
driver.get(url)
use_ISBN = wait_for_element_clickable(driver, "/html/body/form[2]/table/tbody/tr[2]/td[3]/table[2]/tbody/tr[2]/td[1]/table[1]/tbody/tr[2]/td/select/option[8]", 5, By.XPATH).click()
search_input = wait_for_element_clickable(driver, "txtKeyWord", 5, By.NAME)
search_input.send_keys(ISBN)
gogo = wait_for_element_clickable(driver, "btnSearch", 5, By.NAME).click()




table = wait_for_element_clickable(driver, "Table1", 5, By.ID) 
trlist = table.find_elements_by_tag_name('tr')
for row in trlist:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        if tdlist[7].text != "圖書":
            continue
        else:
            tdlist[2].click()
            time.sleep(3)
            table = accurately_find_table_and_read_it(driver, "table.table_1")
            print(table)
            table.drop([0], inplace=True)
            table = organize_columns(table)
            table_lst.append(table)
            back = wait_for_element_clickable(driver, "查詢結果", 1, By.LINK_TEXT).click() 



#all_table = pd.concat(table_lst, axis=0, ignore_index=True)
#all_table.rename(columns={0: '圖書館', 1: '館藏地', 2: '索書號', 3: '館藏狀態', 4: '連結'}, inplace = True)


None


AttributeError: 'NoneType' object has no attribute 'drop'

In [11]:
#敏實科技大學
url = "http://120.105.200.52/xsearch-b.html"
ISBN = 9789861371955

def 敏實科技(org, url, ISBN, driver):
    driver.get(url)
    search_input = wait_for_element_clickable(driver, "DB.IN1", 5, By.NAME)
    search_input.send_keys(ISBN)
    gogo = wait_for_element_clickable(driver, "btn.btn-primary", 5, By.CLASS_NAME).click()

    where = wait_for_element_clickable(driver, "/html/body/table[3]/tbody/tr[2]/td[2]/a", 5, By.XPATH).click()
    time.sleep(3)
    table = accurately_find_table_and_read_it("table", 3)
    table = organize_columns(table)
    return table

None


In [11]:
#中臺科技大學 + 中洲
url = "http://120.107.56.24/isbn1.htm"
ISBN = 9789861371955
org = "中臺科技大學"
http://163.23.234.194/isbn1.htm

def 中臺+中洲_crawler(org, url, driver) :
    driver.get(url)
    search_input = wait_for_element_clickable(driver, "input", 5, By.NAME)
    search_input.send_keys(ISBN)
    gogo = wait_for_element_clickable(driver, "query", 5, By.NAME).click()

    where = wait_for_element_clickable(driver, "body > div > font > font > form > center:nth-child(1) > table > tbody > tr:nth-child(2) > td:nth-child(4) > font > a", 5, By.CSS_SELECTOR).click()
    table = accurately_find_table_and_read_it("body > div > form > center:nth-child(4) > center:nth-child(4) > table", table_position2=3)
    table = organize_columns(table)
    table.drop([0], inplace=True)
    return table

SyntaxError: invalid syntax (<ipython-input-11-ebf53de15958>, line 5)

In [10]:
#要一直點進去的 (改成短網址的，解決登入問題)

def clickclick_crawler(org, url, ISBN, xpath_num, gogo_xpath, xpath_detail, table_xpath, index_lst, driver):
    clickclick_lst = [] 
    into_1_lst = ["馬偕醫學院", "工業技術研究院", "國立清華大學", "國立臺灣美術館", "國立臺灣史前文化博物館", "國立臺灣歷史博物館"]
    onlyonelib = ["國家兩廳院", "臺灣神學研究學院", "明志科技大學", "長庚科技大學", "國立臺灣美術館", "國立臺灣歷史博物館"] #這些沒有分館，print小分區
    try: 
        # 分三類的進入方式
        driver.get(url) 
        if org in into_1_lst: #那種類型的沒辦改網址進進階搜尋QQ
            pro_search = wait_for_element_clickable(driver, "進階查詢", 5, By.LINK_TEXT).click()
        ISBN_xpath = "/html/body/form/table[1]/tbody/tr[2]/td[1]/select/option[" + xpath_num + "]"  #換成ISBN搜尋，xpath_num
        use_ISBN = wait_for_element_clickable(driver, ISBN_xpath, 5, By.XPATH).click()
        search_input = wait_for_element_clickable(driver, "request", 5, By.NAME)
        search_input.send_keys(ISBN) 
        gogo = wait_for_element_clickable(driver, gogo_xpath, 5, By.XPATH).click() #按下確定，gogo_xpath
        click_result = wait_for_element_clickable(driver, "/html/body/form/table[1]/tbody/tr[2]/td[4]/a", 10, By.XPATH).click()

 
        #終於結束前面的輸入可以開始爬蟲了
        try: #暨南有"直接進去書的頁面"的案例，所以先用try避開看看
            where2 = wait_for_element_clickable(driver, "brieftit", 5, By.CLASS_NAME).click() 
        except:
            pass
        if org == "國立暨南國際大學":
            where3_xpath = "/html/body/table[9]/tbody/tr[1]/td[2]/a"
        else: 
            where3_xpath = "/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/" + xpath_detail #按下書在哪裡?，xpath_detail
        where3 = wait_for_element_clickable(driver, where3_xpath, 5, By.XPATH).click()
        table = wait_for_element_clickable(driver, table_xpath, 5, By.XPATH) # 找表格位置，table_xpath
        trlist = table.find_elements_by_tag_name('tr')
        now_url = driver.current_url
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                new_row = [org, tdlist[index_lst[0]].text, tdlist[index_lst[1]].text, tdlist[index_lst[2]].text, now_url]
                clickclick_lst.append(new_row)
                break
    except:
        pass
    
    table = pd.DataFrame(clickclick_lst)
    table.rename(columns={0: '圖書館', 1: '館藏地', 2: '索書號', 3: '館藏狀態', 4: '連結'}, inplace = True)
    return table

ISBN = 9789869255790
print(clickclick_crawler(
            "國立臺灣歷史博物館", "http://lib.moc.gov.tw/F?func=find-b-0&local_base=THM01",
            ISBN, "7", "/html/body/form/table[1]/tbody/tr[9]/td/input", "a[1]", '/html/body/table[9]', [3, 4, 7], driver
        )
    )

         圖書館    館藏地            索書號  館藏狀態  \
0  國立臺灣歷史博物館  一般圖書區  957 4462 2016  仍在館內   

                                                  連結  
0  https://lib.moc.gov.tw/F/QTH34GSPT3FJG57JMN6AG...  


In [2]:
#要一直點進去的(未處理登入問題)
#馬偕醫學院、工研院、清華要補空的behind


def clickclick_crawler(org, url_front, ISBN, url_behind, xpath_num, xpath_detail, table_xpath, driver):
    clickclick_lst = [] 
    ISBN_xpath = "/html/body/table[6]/tbody/tr/td[1]/form/fieldset[1]/select/option[" + xpath_num + "]"
    try:
        # 分三類的進入方式
        if org == "馬偕醫學院" or org == "工研院":
            driver.get(url_front)
            use_ISBN = wait_for_element_clickable(driver, ISBN_xpath, 5, By.XPATH).click()
            search_input = wait_for_element_clickable(driver, "y", 5, By.NAME)
            search_input.send_keys(ISBN)
            gogo = wait_for_element_clickable(driver, "Search", 5, By.NAME).click()
        elif org == "國立暨南國際大學":
            driver.get(url_front)
            backtofirst = wait_for_element_clickable(driver, "/html/body/form/div/div[1]/div/a[2]", 5, By.XPATH)
            
            search_input = wait_for_element_clickable(driver, "request", 5, By.NAME)
            search_input.send_keys(ISBN)
            gogo = wait_for_element_clickable(driver, "/html/body/form/table[1]/tbody/tr[9]/td/input", 5, By.XPATH).click()
            click_result = wait_for_element_clickable(driver, "/html/body/form/table[1]/tbody/tr[3]/td[4]/a", 5, By.XPATH).click()
        else:
            driver.get(url_front + ISBN +url_behind)
        
        where2 = wait_for_element_clickable(driver, "brieftit", 5, By.CLASS_NAME).click() 
        if org == "國立暨南國際大學":
            where3_xpath = "/html/body/table[9]/tbody/tr[1]/td[2]/a"
        else:
            where3_xpath = "/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/" + xpath_detail
        where3 = wait_for_element_clickable(driver, where3_xpath, 5, By.XPATH).click()
        table = wait_for_element_clickable(driver, table_xpath, 5, By.XPATH)
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if org != "工業技術研究院":
                    now_url = driver.current_url
                    new_row = [org, tdlist[2].text, tdlist[4].text, tdlist[7].text, now_url]
                else:
                    new_row = [org, tdlist[2].text, tdlist[4].text, tdlist[8].text, now_url]
                clickclick_lst.append(new_row)
                break
    except:
        pass
    
    return pd.DataFrame(clickclick_lst)

In [ ]:
def primo_finding(driver, org, tcn): #primo爬資訊的def ；#tcn = thelist_class_name
	sub_df_lst = []
	try:
        back = wait_for_element_present(driver, ".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple", 20, By.CSS_SELECTOR)
	except:
		back = None
	if back != None:
		back.click()

	thelist = wait_for_elements_present(driver, tcn, 30, By.CLASS_NAME)
	if tcn == 'md-2-line.md-no-proxy._md': #如果是東吳或銘傳
		thelist = thelist[0:-2]
	else:
		pass

	for row in thelist:
		plist = row.find_elements_by_tag_name("p")
		where = row.find_elements_by_tag_name("h3")
		i = len(where)
		for sth in plist:
			a = sth.find_elements_by_tag_name("span")
			for _ in range(i):
				now_url = driver.current_url
				new_row = [org, where[_].text, a[4].text, a[0].text, now_url]
				sub_df_lst.append(new_row)
				break
			break
	return sub_df_lst

def primo_greendot_finding(driver, org): #primo爬資訊的def
    sub_df_lst = []
    try:
		num = wait_for_element_present(driver, 'EXLLocationTableColumn1', 10, By.CLASS_NAME)
		status = wait_for_element_present(driver, 'EXLLocationTableColumn3', 10, By.CLASS_NAME)
        for i in range(0, len(num)):
            now_url = driver.current_url
            new_row = [org, "圖書館總館", num[i].text, status[i].text, now_url]
            sub_df_lst.append(new_row)
    except:
        pass
	
    return sub_df_lst

In [ ]:
# primo_crawler()
# 臺大|政大|淡大|東吳
def primo_crawler(org, url_front, ISBN ,url_behind, tcn, driver):
    url = url_front + ISBN + url_behind
    primo_lst = []

    try:
        # 進入《館藏系統》頁面
        driver.get(url)

        try: #開始爬蟲
            editions = wait_for_elements_present(driver, 'item-title', 30, By.CLASS_NAME)
            if len(editions) > 1: #如果最外面有兩個版本(默認點進去不會再分版本了啦)(ex.政大 9789861371955)，直接交給下面處理
                pass
            else: #如果最外面只有一個版本，那有可能點進去還有再分，先click進去，再分一個版本跟多個版本的狀況
                time.sleep(5)
                editions[0].click()
                time.sleep(5)
                editions = driver.find_elements_by_class_name('item-title') #這時候是第二層的分版本了！(ex.政大 9789869109321)
                
            try: #先找叉叉確定是不是在最裡層了
                back_check = wait_for_element_present(driver, "md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple", 15, By.CLASS_NAME)
            except:
                back_check = None
            if back_check == None: #多個版本才要再跑迴圈(找不到叉叉代表不在最裡面，可知不是一個版本)
                for i in range(0, len(editions)): #有幾個版本就跑幾次，不管哪一層版本都適用
                    time.sleep(5)
                    into = editions[i].click()
                    if org == "屏東科技大學" or org == "高雄餐旅大學":
                        primo_lst += primo_two_finding(org, tcn, driver)
                    else:
                        primo_lst += primo_finding(org, tcn, driver)
                    try: 
                        back2 = wait_for_element_clickable(driver, "md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple", 15, By.CLASS_NAME)
                    except:
                        back2 = None

            else: #如果只有一個版本(有叉叉的意思)，那前面已經click過了不能再做
                if org == "屏東科技大學" or org == "高雄餐旅大學":
                    primo_lst += primo_two_finding(org, tcn, driver)
                else:
                    primo_lst += primo_finding(org, tcn, driver)
                
        except:
            pass
    except:
        pass
    table = pd.DataFrame(primo_lst)
    table.rename(columns={0: '圖書館', 1: '館藏地', 2: '索書號', 3: '館藏狀態', 4: '連結'}, inplace = True)
    return table

In [ ]:
# primo_greendot_crawler()
# 長庚|中正
def primo_greendot_crawler(driver, org, url_front, ISBN ,url_behind):
    url = url_front + ISBN + url_behind
    primo_greendot_lst = []

    try:
        driver.get(url)
        try: #只有一個版本
            place_click = wait_for_element_clickable(driver, 'exlidResult0-LocationsTab', 15, By.ID)
            primo_greendot_lst += primo_greendot_finding(driver, org)
        except: #有多個版本，所以要點進去再做
            manyeditions = wait_for_element_clickable(driver, 'titleLink', 10, By.ID)
            for i in range(1, 10): #假設有十個版本吧
                try:
                    id = 'exlidResult' + str(i) + '-LocationsTab'
                    place2_click = wait_for_element_clickable(driver, id, 15, By.ID)
                    primo_greendot_lst += primo_greendot_finding(driver, org)
                except:
                    continue
    except:
        pass
    table = pd.DataFrame(primo_greendot_lst)
    table.rename(columns={0: '圖書館', 1: '館藏地', 2: '索書號', 3: '館藏狀態', 4: '連結'}, inplace = True)
    return table

In [ ]:
# primo_two_crawler() 
# 屏科大|高餐
def primo_two_crawler(driver, org, url_front, ISBN ,url_behind):
    url = url_front + ISBN + url_behind
    primo_two_lst = []
    def primo_two_finding(org): #爬資訊的def
        sub_df_lst = []
        try:
            back = wait_for_element_present(driver, ".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple", 20, By.CSS_SELECTOR)
        except:
            back = None
        if back != None:
            back.click()

        similar_xpath = "/html/body/primo-explore/div[3]/div/md-dialog/md-dialog-content/sticky-scroll/prm-full-view/div/div/div[2]/div/div[1]/div[4]/div/prm-full-view-service-container/div[2]/div/prm-opac/md-tabs/md-tabs-content-wrapper/md-tab-content[2]/div/md-content/prm-location-items/div[2]/div[1]/p/span["
        status_xpath = similar_xpath + "1]"
        place_xpath = similar_xpath + "3]"
        num_xpath = similar_xpath + "5]"
        status = wait_for_element_present(driver, status_xpath, 20, By.XPATH)
        place = wait_for_element_present(driver, place_xpath, 20, By.XPATH)
        num = wait_for_element_present(driver, num_xpath, 20, By.XPATH)

        now_url = driver.current_url
        number = num.text.replace("(", "").replace(")", "")
        new_row = [org, place.text, number, status.text, now_url]
        sub_df_lst.append(new_row)

        return sub_df_lst

    try:
        # 進入《館藏系統》頁面
        driver.get(url)
        # 等待＂進階查詢的按鈕＂直到出現：click
        time.sleep(15)

        try: #開始爬蟲
            editions = driver.find_elements_by_class_name('item-title') 
            if len(editions) > 1: #如果最外面有兩x`個版本(默認點進去不會再分版本了啦)(ex.政大 9789861371955)，直接交給下面處理
                pass
            else: #如果最外面只有一個版本，那有可能點進去還有再分，先click進去，再分一個版本跟多個版本的狀況
                time.sleep(2)
                editions[0].click()
                time.sleep(5)
                editions = driver.find_elements_by_class_name('item-title') #這時候是第二層的分版本了！(ex.政大 9789869109321)
                
            try: #先找叉叉確定是不是在最裡層了
                back_check = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back_check = None
            if back_check == None: #多個版本才要再跑迴圈(找不到叉叉代表不在最裡面，可知不是一個版本)
                for i in range(0, len(editions)): #有幾個版本就跑幾次，不管哪一層版本都適用
                    into = editions[i].click()
                    time.sleep(3)
                    primo_two_lst += primo_two_finding(org)
                    try: 
                        back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
                    except:
                        back2 = None

            else: #如果只有一個版本(有叉叉的意思)，那前面已經click過了不能再做
                time.sleep(3)
                primo_two_lst += primo_two_finding(org)
        except:
            pass
    except:
        pass

    table = pd.DataFrame(primo_two_lst)
    table.rename(columns={0: '圖書館', 1: '館藏地', 2: '索書號', 3: '館藏狀態', 4: '連結'}, inplace = True)
    return table

In [30]:
#台中科技大學
org = "台中科技大學"
ISBN = str(9789574672028)
df_lst = []
url = "https://ntit.ent.sirsidynix.net/client/zh_TW/NUTC/search/results?qu=" + ISBN + "&te=ILS"


driver.get(url)
time.sleep(3)
into_first = driver.find_element_by_id("detailLink0").click() #進第一個版本
time.sleep(5)

print(accurately_find_table_and_read_it("#detailItemsDiv0 > div > table", table_position2=0))


            


None


In [22]:
#台中科技大學
org = "台中科技大學"
ISBN = str(9789574672028)
df_lst = []
url = "https://ntit.ent.sirsidynix.net/client/zh_TW/NUTC/search/results?qu=" + ISBN + "&te=ILS"


driver.get(url)
time.sleep(3)
into_first = driver.find_element_by_id("detailLink0").click() #進第一個版本
time.sleep(3)
for i in range(0, 6):
    booktype = driver.find_element_by_class_name("detailItemsTable_ITYPE").text
    if booktype == "電子資源": #不爬是電子資源的
        pass
    else:
        for i in range(0, 6): 
            table_position = "#detailItemsDiv" + str(i) + "> div > table"
            try:
                edition = driver.find_element_by_css_selector(table_position) # 如果那個版本在
                df_lst.append(accurately_find_table_and_read_it(table_position, table_position2=0))
                try:
                    next = driver.find_element_by_class_name("nextArrowRight").click() #按去下一個版本
                except:
                    pass
            except:
                pass
            
print(df_lst)

[None, None, None, None, None, None, None, None]


In [20]:
#台中科技大學
org = "台中科技大學"
ISBN = str(9789574672028)
df_lst = []
url = "https://ntit.ent.sirsidynix.net/client/zh_TW/NUTC/search/results?qu=" + ISBN + "&te=ILS"


driver.get(url)
time.sleep(3)
into_first = driver.find_element_by_id("detailLink0").click() #進第一個版本
time.sleep(3)
for i in range(0, 6):
    booktype = driver.find_element_by_class_name("detailItemsTable_ITYPE").text
    if booktype == "電子資源": #不爬是電子資源的
        pass
    else:
        try:
            html_text = driver.page_source
            dfs = pd.read_html(html_text, encoding="utf-8")
            df_ntit = dfs[4]

            df_lst.append(df_ntit)
            next = driver.find_element_by_class_name("nextArrowRight").click() #按去下一個版本
        except:
            pass

print(df_lst)

[]
